## Model Predictions

In [1]:
# Importing required libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.models import load_model

In [2]:
# Loading all the saved models

model = load_model("models/ann_model.h5")

with open("models/gender_label_encoder.pkl", 'rb') as file:
    gender_label_encoder = pickle.load(file)

with open('models/geography_ohe.pkl', 'rb') as file:
    geography_ohe = pickle.load(file)

with open('models/standard_scaler.pkl', 'rb') as file:
    standard_scaler = pickle.load(file)


In [3]:
# Sample input data for prediction

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
# One Hot Encoding 'Geography'

geography_encoded = geography_ohe.transform([[input_data['Geography']]]).toarray()
geography_encoded_df = pd.DataFrame(geography_encoded, columns = geography_ohe.get_feature_names_out(['Geography']))
geography_encoded_df.head()

/opt/anaconda3/envs/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
# Converting input data to a dataframe

input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
# Encoding the categorical variables

input_data['Gender'] = gender_label_encoder.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
# Combining OHE columns with input data

input_data = pd.concat([input_data.drop("Geography", axis = 1), geography_encoded_df], axis = 1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
# Scaling the data

scaled_input = standard_scaler.transform(input_data)
scaled_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
# Prediction

prediction = model.predict(scaled_input)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
[[0.03823541]]


In [11]:
prediction_probability = prediction[0][0]

In [12]:
if prediction_probability > 0.5:
    print("The custoer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
